In [1]:
import os, pickle, logging, pickle, joblib, sys, warnings
warnings.simplefilter('ignore')
from scipy import stats
import numpy as np
import pandas as pd

import catboost as cb

from sklearn import ensemble, metrics, pipeline, preprocessing, impute, model_selection
from scipy.stats import pearsonr, spearmanr

ML_RAW_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/rawData/"
FS_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/fs_PSugar/"
TEST_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/tesRes/"

In [2]:
feat_col = []
with open(os.path.join(FS_PATH, "cb_rfa_cv_psugar_rmse")) as f:
    for line in f:
        feat_col.append(line.strip())

raw_data = pd.read_table(os.path.join(ML_RAW_PATH, "raw_data_PSugar.txt"), sep="\t", index_col=0)

target_col = ['GZ-PSugar']
raw_data = raw_data.dropna(subset=target_col)

#cat_col_names_raw = ["heading_stage_d", "leaf_blast_rep1", "leaf_blast_rep2", "leaf_blast_average", "GR_D3", "GR_D2", "GR_D1"]
#cat_col_names_raw = ["feat_{}".format(x) for x in cat_col_names_raw]
#cat_col_names = [i for i in feat_col if i in cat_col_names_raw]
#num_col_names = [i for i in feat_col if (i.startswith("feat")) and (i not in cat_col_names)]
#cat_col_num = []
#for item in cat_col_names:
    #cat_col_num.append(feat_col.index(item))

#for col in cat_col_names:
    #raw_data[col] = raw_data[col].astype("str")

kf = model_selection.KFold(n_splits=5, shuffle=True,  random_state=0)
y_test_final, y_pred_final = [], []
for i, (train_index, test_index) in enumerate(kf.split(raw_data)):
    data_train = raw_data.iloc[train_index].copy()
    data_test = raw_data.iloc[test_index].copy()

    scale_tool = preprocessing.StandardScaler()
    scale_tool.fit(data_train.loc[:, feat_col])
    data_train.loc[:, feat_col] = scale_tool.transform(data_train.loc[:, feat_col])
    data_test.loc[:, feat_col] = scale_tool.transform(data_test.loc[:, feat_col])

    train_sel = data_train.sample(frac=0.8, random_state=0)
    val_sel = data_train.drop(train_sel.index).copy()

    X_train = train_sel[feat_col].copy()
    y_train = train_sel[target_col].values.ravel()

    X_val = val_sel[feat_col].copy()
    y_val = val_sel[target_col].values.ravel()

    X_test = data_test[feat_col].copy()
    y_test = data_test[target_col].values.ravel()

    # Initialize CatBoostClassifier
    clf_model = cb.CatBoostRegressor(random_state=0, thread_count=4, loss_function='RMSE')
    # Fit model
    clf_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1, plot=False)

    y_pred = clf_model.predict(X_test)

    y_test_final.append(y_test)
    y_pred_final.append(y_pred)

Learning rate set to 0.039298
0:	learn: 1.9921304	test: 1.9921304	test1: 2.5348545	best: 2.5348545 (0)	total: 47.9ms	remaining: 47.8s
1:	learn: 1.9627813	test: 1.9627813	test1: 2.5103912	best: 2.5103912 (1)	total: 49ms	remaining: 24.4s
2:	learn: 1.9390042	test: 1.9390042	test1: 2.4875128	best: 2.4875128 (2)	total: 50.1ms	remaining: 16.6s
3:	learn: 1.9128444	test: 1.9128444	test1: 2.4638678	best: 2.4638678 (3)	total: 51ms	remaining: 12.7s
4:	learn: 1.8885925	test: 1.8885925	test1: 2.4404049	best: 2.4404049 (4)	total: 52.1ms	remaining: 10.4s
5:	learn: 1.8673782	test: 1.8673782	test1: 2.4261315	best: 2.4261315 (5)	total: 53.2ms	remaining: 8.81s
6:	learn: 1.8478940	test: 1.8478940	test1: 2.4084619	best: 2.4084619 (6)	total: 54.1ms	remaining: 7.68s
7:	learn: 1.8233200	test: 1.8233200	test1: 2.3885594	best: 2.3885594 (7)	total: 55ms	remaining: 6.82s
8:	learn: 1.8053427	test: 1.8053427	test1: 2.3751832	best: 2.3751832 (8)	total: 56ms	remaining: 6.17s
9:	learn: 1.7868720	test: 1.7868720	test1:

In [3]:
res_df = pd.DataFrame(columns=["Model", "Times", "Score", "Type"])
for i in range(len(y_test_final)):
    ## Get score
    #score_pear = pearsonr(y_test_final[i], y_pred_final[i])[0]
    score_spear = spearmanr(y_test_final[i], y_pred_final[i])[0]
    #score_rmse = metrics.mean_squared_error(y_test_final[i], y_pred_final[i], squared=False)
    score_rmse = metrics.root_mean_squared_error(y_test_final[i], y_pred_final[i])
    score_nrmse = score_rmse / np.std(y_test_final[i])

    #res_df.loc[len(res_df)] = ["CB_RFA", i+1, score_pear, "R"]
    res_df.loc[len(res_df)] = ["CB_RFA", i+1, score_spear, "R"]
    res_df.loc[len(res_df)] = ["CB_RFA", i+1, score_rmse, "RMSE"]
    res_df.loc[len(res_df)] = ["CB_RFA", i+1, score_nrmse, "NRMSE"]

with open(os.path.join(TEST_RES_PATH, "PSugar_CB_RFA.pickle"), "wb") as out_f:
    pickle.dump(res_df, out_f)

In [2]:
with open(os.path.join(TEST_RES_PATH, "PSugar_CB_RFA.pickle"), "rb") as f:
    res_df = pickle.load(f)

res_df

,Model,Times,Score,Type
0,CB_RFA,1,0.679870,R
1,CB_RFA,1,1.844721,RMSE
2,CB_RFA,1,0.709347,NRMSE
3,CB_RFA,2,0.741414,R
4,CB_RFA,2,2.100041,RMSE
5,CB_RFA,2,0.772624,NRMSE
6,CB_RFA,3,0.678355,R
7,CB_RFA,3,2.253550,RMSE
8,CB_RFA,3,0.790255,NRMSE
9,CB_RFA,4,0.573207,R


In [3]:
res_df.groupby(["Type"])["Score"].mean()

Type
NRMSE    0.739144
R        0.696484
RMSE     2.019611
Name: Score, dtype: float64